## 1.2: Preprocess Data
Currently, the dataset has four fields:
1. 'text' is a list of a list of tokens(string)
2. 'verb_index' represents the **relative position** of the **predicate verb** in the sentence. (Type: a list of integers)

> Example: He would n't accept anything of value from those he was writing about.

> Verb index: 3, because accept is the fourth word of the sentence (indexing starts from zero).


3. 'srl_label' is the semantic role label of every token. (Type: a list of a list of strings) \\

4. 'word_indices' is the index of every word token (same as howework 1&2, Type: a list of a list of integers)

### **Converting Labels to Numerical Values**
In addition to converting our tokens to vector representations, we need to convert our labels to numerical representations. For example, say we have two labels: "O", "B-ARG0". We could numerically represent these labels in a dictionary as Label 0 and Label 1: `{"O":0, "B-ARG":1}`. The following function is implemented for you, and it encodes the labels present in the dataset. No modifications are necessary.


In [56]:
srl_map = {'O': 0,
           'B-ARG0': 1,
           'I-ARG0': 2,
           'B-ARG1': 3,
           'I-ARG1': 4,
           'B-ARG2': 5,
           'I-ARG2': 6,
           'B-ARGM-LOC': 7,
           'I-ARGM-LOC': 8,
           'B-ARGM-TMP': 9,
           'I-ARGM-TMP': 10}

def encode_srl_category(category_data: List[List[str]])->List[List[int]]:
  """ Encoding SRL category from a list of strings to a list of integers

  Arguments:
    category_data (list(list(str))): SRL categories

  Returns:
    encoded category (list(list(int))): Numerical conversions of SRL categories
  """
  encoded_category = []
  for srl_list in category_data:
    encoded_srl_list = []
    for srl in srl_list:
      try:
        encoded_srl_list.append(srl_map[srl])
      except:
        encoded_srl_list.append(0)
    encoded_category.append(encoded_srl_list)
  return encoded_category

We have also provided the `pad_sents` function, which pads all sentences to be the same length, specifically the length of the longest input sentence, using the provided `pad_token`. Finally, we have provided the `Vocab` class that represents the corpus as a `Vocab` object with several helper functions. These are used to preprocess the training set for you.

In [57]:
def pad_sents(sents, pad_token):
    sents_padded = []

    max_len = max([len(sent) for sent in sents])
    sents_padded = [(sent + ([pad_token] * (max_len - len(sent)))) for sent in sents]

    return sents_padded

class Vocab(object):
    """ Vocabulary, i.e. structure containing either
    src or tgt language terms.
    """
    def __init__(self, word2id=None):
        """ Init Vocab Instance.

        :param word2id: dictionary mapping words 2 indices
        :type word2id: dict[str, int]
        """
        if word2id:
            self.word2id = word2id
        else:
            self.word2id = dict()
            self.word2id['<pad>'] = 0   # Pad Token
            self.word2id['<s>'] = 1 # Start Token
            self.word2id['</s>'] = 2    # End Token
            self.word2id['<unk>'] = 3   # Unknown Token
        self.unk_id = self.word2id['<unk>']
        self.id2word = {v: k for k, v in self.word2id.items()}

    def __getitem__(self, word):
        """ Retrieve word's index. Return the index for the unk
        token if the word is out of vocabulary.

        :param word: word to look up
        :type word: str
        :returns: index of word
        :rtype: int
        """
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        """ Check if word is captured by Vocab.

        :param word: word to look up
        :type word: str
        :returns: whether word is in vocab
        :rtype: bool
        """
        return word in self.word2id

    def __setitem__(self, key, value):
        """ Raise error, if one tries to edit the Vocab directly.
        """
        raise ValueError('vocabulary is readonly')

    def __len__(self):
        """ Compute number of words in Voca.

        :returns: number of words in Vocab
        :rtype: int
        """
        return len(self.word2id)

    def __repr__(self):
        """ Representation of Vocab to be used
        when printing the object.
        """
        return 'Vocabulary[size=%d]' % len(self)

    def add(self, word):
        """ Add word to Vocab, if it is previously unseen.

        :param word: to add to Vocab
        :type word: str
        :returns: index that the word has been assigned
        :rtype: int
        """
        if word not in self:
            wid = self.word2id[word] = len(self)
            self.id2word[wid] = word
            return wid
        else:
            return self[word]

    def words2indices(self, sents):
        """ Convert list of words or list of sentences of words
        into list or list of list of indices.

        :param sents: sentence(s) in words
        :type sents: Union[List[str], List[List[str]]]
        :returns: sentence(s) in indices
        :rtype: Union[List[int], List[List[int]]]
        """
        if type(sents[0]) == list:
            return [[self[w] for w in s] for s in sents]
        else:
            return [self[w] for w in sents]

    def indices2words(self, word_ids):
        """ Convert list of indices into words.

        :param word_ids: list of word ids
        :type word_ids: List[int]
        :returns: list of words
        :rtype: List[Str]
        """
        return [self.id2word[w_id] for w_id in word_ids]

    def to_input_tensor(self, sents: List[List[str]], device: torch.device) -> torch.Tensor:
        """ Convert list of sentences (words) into tensor with necessary padding for
        shorter sentences.

        :param sents: list of sentences (words)
        :type sents: List[List[str]]
        :param device: Device on which to load the tensor, ie. CPU or GPU
        :type device: torch.device
        :returns: Sentence tensor of (batch_size, max_sentence_length)
        :rtype: torch.Tensor
        """
        word_ids = self.words2indices(sents)
        sents_t = pad_sents(word_ids, self['<pad>'])
        sents_var = torch.tensor(sents_t, dtype=torch.long, device=device)
        return sents_var

    @staticmethod
    def from_corpus(corpus, size, remove_frac=None, freq_cutoff=None):
        """ Given a corpus construct a Vocab.

        :param corpus: corpus of text produced by read_corpus function
        :type corpus: List[str]
        :param freq_cutoff: if word occurs n < freq_cutoff times, drop the word
        :type freq_cutoff: int
        :returns: Vocab instance produced from provided corpus
        :rtype: Vocab
        """
        vocab_entry = Vocab()
        word_freq = Counter(chain(*corpus))
        if freq_cutoff is None:
            freq_cutoff = 1
        valid_words = [w for w, v in word_freq.items() if v >= freq_cutoff]
        print('number of word types: {}, number of word types w/ frequency >= {}: {}'
              .format(len(word_freq), freq_cutoff, len(valid_words)))
        top_words = sorted(valid_words, key=lambda word: word_freq[word], reverse=True)
        if remove_frac is not None:
            size = len(top_words) - int(remove_frac * len(top_words))
            top_words = top_words[:size]
            print(f'number of unique words retained with remove_frac={remove_frac}: {len(top_words)}')
        for word in top_words:
            vocab_entry.add(word)
        return vocab_entry

    @staticmethod
    def from_subword_list(subword_list):
        """Given a list of subwords, construct the Vocab.

        :param subword_list: list of subwords in corpus
        :type subword_list: List[str]
        :returns: Vocab instance produced from provided list
        :rtype: Vocab
        """
        vocab_entry = Vocab()
        for subword in subword_list:
            vocab_entry.add(subword)
        return vocab_entry


# **Part 2: LSTM Encoder Model**

In [58]:
# Setting seed ***DO NOT MODIFY***
torch.manual_seed(123)

print('initialize train vocabulary ..')
src_vocab = Vocab.from_corpus(train['text'], 20000, remove_frac=0.3)

initialize train vocabulary ..
number of word types: 19082, number of word types w/ frequency >= 1: 19082
number of unique words retained with remove_frac=0.3: 13358


In [59]:
train_data = list(zip(train['text'],train['verb_index'],encode_srl_category(train['srl_frames'])))
val_data = list(zip(val['text'], val['verb_index'], encode_srl_category(val['srl_frames'])))

In [60]:
# Lambda to switch to GPU if available
get_device = lambda : "cuda:0" if torch.cuda.is_available() else "cpu"

## 2.1 Implementation

Your first task is to **implement the model below by finishing the #TODOs**.


<a name="l2"></a>

In [61]:
from torch import lstm
class LSTMTagger(nn.Module):
    def __init__(self, src_vocab, embed_dim,
    hidden_dim, output_dim, vocab_size, num_layers=1):
        '''
        src_vocab: vocabulary of inputs (Class Vocab)
        embed_dim: dimension of word embedding
        hidden_dim: dimension of hidden layer
        output_dim: dimension of tagset_size
        vocab_size: vocabulary size
        num_layers: number of LSTM layers
        '''
        super(LSTMTagger, self).__init__()
        self.src_vocab = src_vocab
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.embed_dim = embed_dim

        ### TODO : Initialize three linear layers:
              # 1. A word embedding layer
              # 2. A LSTM layer
              # 3. An output linear layer
        ### TODO : Initialize logsoftmax
        self.source_embedding = nn.Embedding(len(src_vocab), embed_dim, padding_idx=src_vocab['<pad>'])
        #STAFF: put sequnece in LSTM and if iterate through tokens use LSTM cell (not do it here)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, bidirectional=False, batch_first=True)
        self.linear = nn.Linear((hidden_dim+hidden_dim)*num_layers, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def compute_Loss(self, criterion, predicted_vector, gold_label):
        loss = 0
        for n in range(len(predicted_vector)): # batch size
            loss += criterion(predicted_vector[n], gold_label[n])
        return loss

    def forward(self, source: List[List[str]], verb_indices:List[int]):
        ### GOALS : Write the forward function such that it processes sentences.
        ### GOALS : Return output of the logsoftmax across all time steps

        #pad input sentences and conver to word index
        source_padded = self.src_vocab.to_input_tensor(source, device=get_device())
        batch_size = source_padded.shape[0]
        time_steps = source_padded.shape[1]

        #set up outputs dimension


        #TODO1: Convert word index to embedding

        #TODO2: Pass inputs to the lstm layer

        #TODO3: Get hidden state of verb in the sentence

        #TODO4: Iterate over the time dimension:
        #       - Concatenate verb hidden state to the hidden layer output of every token
        #       - Predict SRL tag distribution with output layer and logsoftmax
        inputs = self.source_embedding(source_padded)

        hidden_states, _ = self.lstm(inputs)

        output = torch.zeros(batch_size, time_steps, self.output_dim).to(get_device())

        verb_embeddings = torch.zeros(batch_size, self.hidden_dim).to(get_device())

        for i in range(batch_size):
          verb_embeddings[i, :] = hidden_states[i][verb_indices[i]]

        for t in range(time_steps):
            y_t = self.softmax(self.linear(torch.cat((hidden_states[:, t, :], verb_embeddings), dim=1)))
            output[:, t, :] = y_t

        return output


    def load_model(self, save_path):
        saved_model = torch.load(save_path)
        self.load_state_dict(saved_model.state_dict())

    def save_model(self, save_path):
        torch.save(self, save_path)


In [62]:
def batch_iter(data, batch_size, shuffle=False):
    """ Yield batches of input sentence, verb indices, target output labels
    :param data: list of tuples containing source and target sentence. ie.
        (list of (src_sent, tgt_sent))
    :type data: List[Tuple[List[str], List[str], List[str]]]
    :param batch_size: batch size
    :type batch_size: int
    :param shuffle: whether to randomly shuffle the dataset
    :type shuffle: boolean
    """
    batch_num = math.ceil(len(data) / batch_size)
    index_array = list(range(len(data)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size]
        examples = [data[idx] for idx in indices]

        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
        srl = [e[0] for e in examples]
        verb_index = [e[1] for e in examples]
        target = [e[2] for e in examples]

        yield srl, verb_index, target

In [63]:
# Setting seed ***DO NOT MODIFY***
torch.manual_seed(123)

def evaluation(model, val_data, optimizer, criterion, batch_size=64):
  model.eval()
  loss = 0
  correct = 0
  total = 0
  batch = 0
  for (input_batch, verb_indices, expected_out) in tqdm(batch_iter(val_data, batch_size=batch_size, shuffle=True)):
    output = model.forward(input_batch, torch.tensor(verb_indices).to(get_device()))
    total += output.size()[0] * output.size()[1]
    _, predicted = torch.max(output, 2)
    expected_out = torch.tensor(pad_sents(expected_out,0))
    correct += (expected_out.to("cpu") == predicted.to("cpu")).cpu().numpy().sum()

    loss += model.compute_Loss(criterion, output.to("cpu"), expected_out.to("cpu"))
    batch += 1
  loss /= batch
  print("Validation Loss: " + str(loss.item()))
  print("Validation Accuracy: " + str(correct/total))
  print()
  return loss.item()

In [64]:
# Setting seed ***DO NOT MODIFY***
torch.manual_seed(123)

def train_epoch(model, train_data, optimizer, criterion,batch_size=64):
  model.train()
  total = 0
  batch = 0
  total_loss = 0
  correct = 0
  for (input_batch, verb_indices, expected_out) in tqdm(batch_iter(train_data, batch_size=batch_size, shuffle=True)):
    optimizer.zero_grad()
    batch += 1
    output = model.forward(input_batch, torch.tensor(verb_indices).to(get_device()))
    total += output.size()[0] * output.size()[1]
    _, predicted = torch.max(output, 2)

    expected_out = torch.tensor(pad_sents(expected_out,0))
    correct += (expected_out.to("cpu") == predicted.to("cpu")).cpu().numpy().sum()

    loss = model.compute_Loss(criterion, output.to("cpu"), expected_out.to("cpu"))
    total_loss += loss.item()
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 1)
    optimizer.step()
  print("Loss: " + str(total_loss/batch))
  print("Training Accuracy: " + str(correct/total))
  return total_loss/batch

In [65]:
# Setting seed ***DO NOT MODIFY***
torch.manual_seed(123)

def train_and_evaluate(number_of_epochs, model, train_data, val_data, criterion, min_loss=0, lr=.01):
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=.9)
  loss_values = [[],[]]
  for epoch in trange(number_of_epochs, desc="Epochs"):
    cur_loss = train_epoch(model, train_data, optimizer, criterion)
    loss_values[0].append(cur_loss)
    cur_loss_val = evaluation(model, val_data, optimizer, criterion)
    loss_values[1].append(cur_loss_val)
    if cur_loss <= min_loss: return loss_values
  return loss_values

**Train your LSTM Encoder Model** with the following cell:

## 2.1 Get Entity level F1 score on the validation set

Run the cells below to calculate your F1 score on the validation set (no modifications needed):

In [67]:
def format_output_labels(token_labels, token_indices):
    """
    Returns a dictionary that has the labels (ARG0,ARG1,ARG2,TMP,LOC) as the keys,
    with the associated value being the list of entities predicted to be of that key label.
    Each entity is specified by its starting and ending position indicated in [token_indices].

    :parameter token_labels: A list of token labels
    :type token_labels: List[String]
    :parameter token_indices: A list of token indices (taken from the dataset)
                              corresponding to the labels in [token_labels].
    :type token_indices: List[int]
    """
    label_dict = {"ARG0":[], "ARG1":[], "ARG2":[], "LOC":[],"TMP":[]}
    prev_label = 'O'
    start = token_indices[0]
    for idx, label in enumerate(token_labels):
      curr_label = label.split('-')[-1]
      if label.startswith('B-') or curr_label != prev_label:
        if prev_label != 'O':
          label_dict[prev_label].append((start, token_indices[idx-1]))
        if curr_label != 'O':
          start = token_indices[idx]
        else:
          start = None

      prev_label = curr_label

    if start is not None and prev_label != 'O':
      label_dict[prev_label].append((start, token_indices[idx]))
    return label_dict

In [68]:
# Code for mean F1

import numpy as np

def mean_f1(y_pred_dict, y_true_dict):
    F1_lst = []
    for key in y_true_dict:
        TP, FN, FP = 0, 0, 0
        num_correct, num_true = 0, 0
        preds = y_pred_dict[key]
        trues = y_true_dict[key]
        for true in trues:
            num_true += 1
            if true in preds:
                num_correct += 1
            else:
                continue
        num_pred = len(preds)
        if num_true != 0:
            if num_pred != 0 and num_correct != 0:
                R = num_correct / num_true
                P = num_correct / num_pred
                F1 = 2*P*R / (P + R)
            else:
                F1 = 0      # either no predictions or no correct predictions
        else:
            continue
        F1_lst.append(F1)
    return np.mean(F1_lst)

In [69]:
#get validation output
inv_srl_map={srl_map[key]:key for key in srl_map}
val_predict = []
val_true = []
val_idx = []

for idx in range(len(val_data)):
  out = lstm.forward([val_data[idx][0]], torch.tensor([val_data[idx][1]]))
  _, predicted = torch.max(out, 2)

  len_sent = len(val_data[idx][0])
  result = predicted.cpu().numpy()[0]

  for t in range(len_sent):
    val_predict.append(inv_srl_map[result[t]])
    val_true.append(inv_srl_map[val_data[idx][2][t]])

  val_idx.extend(val['words_indices'][idx])

In [70]:
#get validation score
y_pred_dict = format_output_labels(val_predict, val_idx)
y_true_dict = format_output_labels(val_true, val_idx)

print(str(mean_f1(y_pred_dict, y_true_dict)))

0.29469556719567025


# **Part 3: Encoder-Decoder Model**

## 3.1 Preprocess the data

The code here converts raw dataset to the input and output format mentioned above. No modifications necessary.

In [24]:
srl_frames = ["ARGM-TMP", "ARG0", "ARG1", "ARG2", "ARGM-LOC"]
def get_srl_frames_indices(token_labels, token_indices):
    label_dict = {"ARGM-TMP":[], "ARG0":[], "ARG1":[], "ARG2":[], "ARGM-LOC":[]}
    prev_label = 'O'
    start = token_indices[0]
    for idx, label in enumerate(token_labels):
      curr_label = '-'.join(label.split('-')[1:]) if label != 'O' else 'O'
      if label.startswith("B-") or (curr_label != prev_label and curr_label != "O"):
        if prev_label != "O":
          label_dict[prev_label].append((start, token_indices[idx-1]))
        start = token_indices[idx]
      elif label == "O" and prev_label != "O":
        label_dict[prev_label].append((start, token_indices[idx-1]))
        start = None
      prev_label = curr_label
    if start is not None and prev_label != 'O':
      label_dict[prev_label].append((start, token_indices[idx-1]))
    return label_dict

In [25]:
SEPT = 'SEP_T'
def generate_source_corpus(source_text: List[List[str]], source_verb: List[int]):
  assert len(source_text) == len(source_verb)
  return [[source_text[i][source_verb[i]]]+ [SEPT] + [token for token in source_text[i]] + [arg] for i in range(len(source_text)) for arg in srl_frames]

In [26]:
def generate_target_corpus(source_text: List[List[str]], source_verb: List[int], source_srl: List[List[str]], source_indices: List[List[str]]):
  assert len(source_text) == len(source_verb)
  assert len(source_text) == len(source_srl)

  ans = []
  for i in range(len(source_text)):
    text = source_text[i]
    verb = text[source_verb[i]]
    srl = source_srl[i]
    indices = source_indices[i]
    indice_start = indices[0]
    label_dict = get_srl_frames_indices(srl, indices)
    for key in label_dict.keys():
      arg_lst = []
      for arg_idx in label_dict[key]:
        arg_lst += text[(arg_idx[0] - indice_start):(arg_idx[1]- indice_start +1)]
      ans += [['<s>'] +[token for token in arg_lst] + ['</s>']]

  return ans

The following cell builds a combined vocab dictionary for input (source) and output (target). Both will share the same vocabulary.

In [27]:
train_src_corpus = generate_source_corpus(train['text'], train['verb_index'])
train_tgt_corpus = generate_target_corpus(train['text'], train['verb_index'], train['srl_frames'], train['words_indices'])
train_data = list(zip(train_src_corpus, train_tgt_corpus))

val_src_corpus = generate_source_corpus(val['text'], val['verb_index'])
val_tgt_corpus = generate_target_corpus(val['text'], val['verb_index'], val['srl_frames'], val['words_indices'])
val_data = list(zip(val_src_corpus, val_tgt_corpus))

#generate src_vocab
src_vocab = Vocab.from_corpus(np.array(train_src_corpus + val_src_corpus+train_tgt_corpus+val_tgt_corpus), 20000, remove_frac=0.3)
tgt_vocab = src_vocab

number of word types: 19871, number of word types w/ frequency >= 1: 19871
number of unique words retained with remove_frac=0.3: 13910


<ipython-input-27-2a2715392125>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  src_vocab = Vocab.from_corpus(np.array(train_src_corpus + val_src_corpus+train_tgt_corpus+val_tgt_corpus), 20000, remove_frac=0.3)


In [29]:
import gensim.downloader as api
model= api.load("glove-wiki-gigaword-300")
torch.manual_seed(1)
#obtain src_embeddings
src_embeddings = []
for i, word in enumerate(src_vocab.word2id.keys()):
    try:
        src_embeddings.append(model[word])
    except:
        src_embeddings.append(torch.rand(300))

src_embeddings = np.stack(src_embeddings, 0)
src_embeddings = torch.from_numpy(src_embeddings)

[==================================================] 100.0% 376.1/376.1MB downloaded


In [30]:
Hypothesis = namedtuple('Hypothesis', ['value', 'score'])

<a name="l4"></a>
## 3.2 Implementation
Your next task is to **implement the Encoder-Decoder model  by finishing the #TODOs**.

In [31]:
class Encoder(nn.Module):
    def __init__(self, embed_size, hidden_size, source_embeddings):
        """
        """
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.embedding = source_embeddings


        ### YOUR CODE HERE (~3 Lines)
        ### TODO - Initialize the following variables:
        ###     self.encoder (Bidirectional LSTM with bias)
        ###     self.h_projection (Linear Layer with bias),called W_{h} above.
        ###     self.c_projection (Linear Layer with bias),called W_{c} above.
        self.encoder = nn.LSTM(embed_size, hidden_size, num_layers=1, bidirectional=True, batch_first = True)
        self.h_projection = nn.Linear(2 * hidden_size, hidden_size)
        self.c_projection = nn.Linear(2 * hidden_size, hidden_size)

    def forward(self, source_padded: torch.Tensor, source_lengths: List[int]) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:

        ### YOUR CODE HERE
        ### TODO:
        ###     1. Construct Tensor `X` by embedding the input. The result should have shape (b, src_len, e)
        ###         b = batch size, src_len = maximum source sentence length, e = embedding size. Note
        ###         that there is no initial hidden state or cell for the decoder.
        ###         Note: you should study the equations/mathematical definitions above to determine
        ###         what some of these values should be.  The same holds throughout.
        ###     2. Compute `enc_hiddens`, `last_hidden`,  `last_cell_state` by applying the LSTM encoder to `X`.
        ###     3. Compute
        ###         - `init_decoder_hidden`:
        ###             `last_hidden` is a tensor shape (2, b, h). The first dimension corresponds to forwards and backwards.
        ###             Concatenate the forwards and backwards tensors to obtain a tensor shape (b, 2*h).
        ###             Apply the h_projection layer to this in order to compute init_decoder_hidden.
        ###             This is h_0^{dec} in above in the writeup. Here b = batch size, h = hidden size
        ###         - `init_cell_hidden`:
        ###             `last_cell_state` is a tensor shape (2, b, h). The first dimension corresponds to forwards and backwards.
        ###             Concatenate the forwards and backwards tensors to obtain a tensor shape (b, 2*h).
        ###             Apply the c_projection layer to this in order to compute init_decoder_hidden.
        ###             This is c_0^{dec} in above in the writeup. Here b = batch size, h = hidden size

        enc_hiddens, dec_init_state = None, None

        X = self.embedding(source_padded)
        X = nn.utils.rnn.pack_padded_sequence(X, source_lengths, batch_first=True)
        enc_hiddens, (last_hidden, cell_state) = self.encoder(X)

        (enc_hiddens, _) = nn.utils.rnn.pad_packed_sequence(enc_hiddens, batch_first=True)

        before_projection = torch.cat((last_hidden[0], last_hidden[1]), dim=-1)
        init_decoder_hidden = self.h_projection(before_projection)

        concat_cell_states = torch.cat((cell_state[0], cell_state[1]), dim=-1)
        cell_state = self.c_projection(concat_cell_states)

        dec_init_state = (init_decoder_hidden, cell_state)

        return enc_hiddens, dec_init_state

In [41]:
class Decoder(nn.Module):
    def __init__(self, embed_size, hidden_size, target_embedding, device):
        """
        """
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.device = device
        self.embedding = target_embedding
        output_vocab_size = self.embedding.weight.size(0)
        self.softmax = nn.Softmax(dim=1)
        self.att_projection = nn.Linear(in_features=self.hidden_size * 2,
                                        out_features=self.hidden_size,
                                        bias=False)

        ### YOUR CODE HERE (~3 lines)
        ###     self.decoder (LSTM Cell with bias)
        ###     self.combined_output_projection (Linear Layer with no bias), called W_{v} above.
        ###     self.target_vocab_projection (Linear Layer with no bias), called W_{target} above.
        ### You may find some of these functions useful:
        ###     LSTM Cell:
        ###     https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html
        self.decoder = nn.LSTMCell(input_size=embed_size+self.hidden_size,
                                   hidden_size=self.hidden_size,
                                   bias=True)
        self.att_projection = nn.Linear(in_features=self.hidden_size * 2,
                                        out_features=self.hidden_size,
                                        bias=False)
        self.combined_output_projection = nn.Linear(in_features=self.hidden_size * 3,
                                                    out_features=self.hidden_size,
                                                    bias=False)
        self.softmax = nn.Softmax(dim=1)
        output_vocab_size = self.embedding.weight.size(0)
        self.target_vocab_projection = nn.Linear(in_features=self.hidden_size,
                                                 out_features=output_vocab_size,
                                                 bias=False)

    def forward(self, enc_hiddens: torch.Tensor,
                dec_init_state: torch.Tensor, target_padded: torch.Tensor) -> torch.Tensor:
        # Chop off the <END> token for max length sentences.
        target_padded = target_padded[:,:-1]

        dec_state = dec_init_state

        # Initialize previous combined output vector o_{t-1} as zero
        batch_size = enc_hiddens.size(0)
        o_prev = torch.zeros(batch_size, self.hidden_size, device=self.device)

        # Initialize a list we will use to collect the combined output o_t on each step
        combined_outputs = []


        ### YOUR CODE HERE
        ### TODO:
        ###     1. Construct tensor `Y` by embedding the target sentences. The result should have shape (b, tgt_len, e)
        ###         where b = batch size, tgt_len = maximum target sentence length, e = embedding size.
        ###     2. Construct enc_hiddens_proj by using self.att_projection to project enc_hiddens to shape (b, src_len, h)
        ###     3. Iterate over the time dimension of Y.
        ###         Within the loop, this will give you Y_t of shape (b, 1, e) where b = batch size, e = embedding size.
        ###             - Squeeze Y_t into a tensor of dimension (b, e).
        ###             - Construct Ybar_t by concatenating Y_t with o_prev on their last dimension
        ###             - Use the step function to compute the the Decoder's next (cell, state) values
        ###               as well as the new combined output o_t.
        ###             - Append o_t to combined_outputs
        ###             - Update o_prev to the new o_t.
        ###     4. Use torch.stack to convert combined_outputs from a list length tgt_len of
        ###         tensors shape (b, h), to a single tensor shape (b, tgt, h)
        ###         where tgt_len = maximum target sentence length, b = batch size, h = hidden size.
        ###
        ### Note:
        ###    - When using the squeeze() function make sure to specify the dimension you want to squeeze
        ###      over. Otherwise, you will remove the batch dimension accidentally.
        ###
        ### You may find some of these functions useful:
        ###     Zeros Tensor:
        ###         https://pytorch.org/docs/stable/torch.html#torch.zeros
        ###     Tensor Dimension Squeezing:
        ###         https://pytorch.org/docs/stable/torch.html#torch.squeeze
        ###     Tensor Concatenation:
        ###         https://pytorch.org/docs/stable/torch.html#torch.cat


        enc_hiddens_proj = self.att_projection(enc_hiddens)
        Y = self.embedding(target_padded)
        for Y_t in torch.split(Y, 1, dim=1):
            Ybar_t = torch.cat((torch.squeeze(Y_t, 1), o_prev), 1)
            dec_state, o_t = self.step(Ybar_t, dec_state, enc_hiddens, enc_hiddens_proj)
            combined_outputs.append(o_t)
            o_prev = o_t


        combined_outputs = torch.stack(combined_outputs, 1)

        return combined_outputs




    def step(self, Ybar_t: torch.Tensor,
            dec_state: Tuple[torch.Tensor, torch.Tensor],
            enc_hiddens: torch.Tensor,
            enc_hiddens_proj: torch.Tensor) -> Tuple[Tuple, torch.Tensor, torch.Tensor]:
        """ Compute one forward step of the LSTM decoder, including the attention computation.

        :param Ybar_t: Concatenated Tensor of [Y_t o_prev], with shape (b, e + h). The input for the decoder,
                                where b = batch size, e = embedding size, h = hidden size.
        :type Ybar_t: torch.Tensor
        :param dec_state: Tensors with shape (b, h), where b = batch size, h = hidden size.
                Tensor is decoder's prev hidden state
        :type dec_state: torch.Tensor
        :param enc_hiddens: Encoder hidden states Tensor, with shape (b, src_len, h * 2), where b = batch size,
                                    src_len = maximum source length, h = hidden size.
        :type enc_hiddens: torch.Tensor
        :param enc_hiddens_proj: Encoder hidden states Tensor, projected from (h * 2) to h. Tensor is with shape (b, src_len, h),
                                    where b = batch size, src_len = maximum source length, h = hidden size.
        :type enc_hiddens_proj: torch.Tensor

        :returns dec_state: Tensors with shape (b, h), where b = batch size, h = hidden size.
                Tensor is decoder's new hidden state
        returns combined_output: Combined output Tensor at timestep t, shape (b, h), where b = batch size, h = hidden size.
        """

        ### TODO 2(Attention Step):
        ###     1. Use dot product to calculate similarity between enc_hiddens_proj and dec_hidden,
        ###        and then take softmax (this is the attention weight alpha_t)
        ###     2. Dot product attention weight with enc_hiddens to get weighted context embedding a_t
        ###     3. U_t = Concate dec_hidden and a_t

        ### TODO 3:
        ###     1. Apply the combined output projection layer to U_t to compute tensor V_t
        # 1
        dec_state = self.decoder(Ybar_t, dec_state)
        # 2
        (dec_hidden, dec_cell) = dec_state

        #b x src_len x h @ b x h x 1 = b x src_len x 1
        e_t = enc_hiddens_proj @ dec_hidden.unsqueeze(2)

        e_t = e_t.squeeze(2)

        alpha_t = self.softmax(e_t)
        # 2 (b, 1, src_len) x (b, src_len, 2h) -> (b, 1, 2h) -> (b, 2h)
        a_t = (alpha_t.unsqueeze(1) @ enc_hiddens).squeeze(1)
        # 3
        U_t = torch.cat((dec_hidden, a_t), 1)
        # 4
        V_t = self.combined_output_projection(U_t)
        # 5
        #O_t = torch.tanh(V_t)

        combined_output = V_t
        return dec_state, combined_output

The following class puts together the LSTM Encoder and Decoder with several helper methods that allow the model to generate outputs. No modifications are necessary.

In [42]:
class SRL(nn.Module):
    """ Simple Semantic Role Labelling Model with:
        - Bidrectional LSTM Encoder
        - Unidirection LSTM Decoder
    """
    def __init__(self, embed_size, hidden_size, src_vocab, tgt_vocab, device=torch.device("cpu"), pretrained_source=None,pretrained_target=None,):
        """ Init SRL Model.

        :param embed_size: Embedding size (dimensionality)
        :type embed_size: int
        :param hidden_size: Hidden Size, the size of hidden states (dimensionality)
        :type hidden_size: int
        :param src_vocab: Vocabulary object containing src language
        :type src_vocab: Vocab
        :param tgt_vocab: Vocabulary object containing tgt language
        :type tgt_vocab: Vocab
        :param device: torch device to put all modules on
        :type device: torch.device
        :param pretrained_source: Matrix of pre-trained source word embeddings
        :type pretrained_source: Optional[torch.Tensor]
        :param pretrained_target: Matrix of pre-trained target word embeddings
        :type pretrained_target: Optional[torch.Tensor]
        """
        super(SRL, self).__init__()
        self.device=device
        self.embed_size = embed_size
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        src_pad_token_idx = src_vocab['<pad>']
        tgt_pad_token_idx = tgt_vocab['<pad>']
        self.source_embedding = nn.Embedding(len(src_vocab), embed_size, padding_idx=src_pad_token_idx)

        with torch.no_grad():
            if pretrained_source is not None:
                self.source_embedding.weight.data = pretrained_source
                self.source_embedding.weight.requires_grad = False

        self.target_embedding = self.source_embedding
        self.hidden_size = hidden_size

        # default values
        self.encoder = Encoder(
            embed_size=embed_size,
            hidden_size=hidden_size,
            source_embeddings=self.source_embedding,
        )
        self.decoder = Decoder(
            embed_size=embed_size,
            hidden_size=hidden_size,
            target_embedding=self.target_embedding,
            device=self.device,
        )


    def forward(self, source: List[List[str]], target: List[List[str]]) -> torch.Tensor:
        """ Take a mini-batch of source and target sentences, compute the log-likelihood of
        target sentences under the language models learned by the SRL system.

        :param source: list of source sentence tokens
        :type source: List[List[str]]
        :param target: list of target sentence tokens, wrapped by `<s>` and `</s>`
        :type target: List[List[str]]
        :returns scores: a variable/tensor of shape (b, ) representing the
                                    log-likelihood of generating the gold-standard target sentence for
                                    each example in the input batch. Here b = batch size.
        :rtype: torch.Tensor
        """
        # Compute sentence lengths
        source_lengths = [len(s) for s in source]

        # Convert list of lists into tensors
        source_padded = self.src_vocab.to_input_tensor(source, device=self.device)
        target_padded = self.tgt_vocab.to_input_tensor(target, device=self.device)

        ###     Run the network forward:
        ###     1. Apply the encoder to `source_padded` by calling `self.encode()`
        ###     2. Apply the decoder to compute combined-output by calling `self.decode()`
        ###     3. Compute log probability distribution over the target vocabulary using the
        ###        combined_outputs returned by the `self.decode()` function.

        enc_hiddens, dec_init_state = self.encode(source_padded, source_lengths)
        combined_outputs = self.decode(enc_hiddens, dec_init_state, target_padded)
        P = F.log_softmax(self.decoder.target_vocab_projection(combined_outputs), dim=-1)

        # Zero out, probabilities for which we have nothing in the target text
        target_masks = (target_padded != self.tgt_vocab['<pad>']).float()

        # Compute log probability of generating true target words
        target_gold_words_log_prob = torch.gather(P, index=target_padded[:,1:].unsqueeze(-1), dim=-1).squeeze(-1) * target_masks[:,1:]
        scores = target_gold_words_log_prob.sum(dim=1)
        return scores


    def encode(self, source_padded: torch.Tensor, source_lengths: List[int]) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """ Apply the encoder to source sentences to obtain encoder hidden states.
            Additionally, take the final states of the encoder and project them to obtain initial states for decoder.

        :param source_padded: Tensor of padded source sentences with shape (src_len, b), where
            b = batch_size, src_len = maximum source sentence length. Note that these have
            already been sorted in order of longest to shortest sentence.
        :type source_padded: torch.Tensor
        :param source_lengths: List of actual lengths for each of the source sentences in the batch
        :type source_lengths: List[int]
        :returns: Tuple of two items. The first is Tensor of hidden units with shape (b, src_len, h*2),
            where b = batch size, src_len = maximum source sentence length, h = hidden size. The second is
            Tuple of tensors representing the decoder's initial hidden state and cell.
        :rtype: Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]
        """
        return self.encoder(source_padded, source_lengths)


    def decode(self, enc_hiddens: torch.Tensor, dec_init_state: Tuple[torch.Tensor, torch.Tensor],
               target_padded: torch.Tensor) -> torch.Tensor:
        """Compute combined output vectors for a batch.

        @param enc_hiddens (Tensor): Hidden states (b, src_len, h*2), where
                                     b = batch size, src_len = maximum source sentence length, h = hidden size.
        @param dec_init_state (tuple(Tensor, Tensor)): Initial state and cell for decoder
        @param target_padded (Tensor): Gold-standard padded target sentences (tgt_len, b), where
                                       tgt_len = maximum target sentence length, b = batch size.

        @returns combined_outputs (Tensor): combined output tensor  (tgt_len, b,  h), where
                                        tgt_len = maximum target sentence length, b = batch_size,  h = hidden size
        """
        return self.decoder(enc_hiddens, dec_init_state, target_padded)


    def beam_search(self, src_sent: List[str], beam_size: int=5, max_decoding_time_step: int=70) -> List[Hypothesis]:
        """ Given a single source sentence, perform beam search, yielding translations in the target language.
        @param src_sent (List[str]): a single source sentence (words)
        @param beam_size (int): beam size
        @param max_decoding_time_step (int): maximum number of time steps to unroll the decoding RNN
        @returns hypotheses (List[Hypothesis]): a list of hypothesis, each hypothesis has two fields:
                value: List[str]: the decoded target sentence, represented as a list of words
                score: float: the log-likelihood of the target sentence
        """
        src_sents_var = self.src_vocab.to_input_tensor([src_sent], self.device)

        src_encodings, dec_init_vec = self.encode(src_sents_var, [len(src_sent)])
        src_encodings_att_linear = self.decoder.att_projection(src_encodings)

        h_tm1 = dec_init_vec
        att_tm1 = torch.zeros(1, self.hidden_size, device=self.device)

        eos_id = self.tgt_vocab['</s>']

        hypotheses = [['<s>']]
        hyp_scores = torch.zeros(len(hypotheses), dtype=torch.float, device=self.device)
        completed_hypotheses = []

        t = 0
        while len(completed_hypotheses) < beam_size and t < max_decoding_time_step:
            t += 1
            hyp_num = len(hypotheses)

            exp_src_encodings = src_encodings.expand(hyp_num,
                                                     src_encodings.size(1),
                                                     src_encodings.size(2)).to(self.device)

            exp_src_encodings_att_linear = src_encodings_att_linear.expand(hyp_num,
                                                                           src_encodings_att_linear.size(1),
                                                                           src_encodings_att_linear.size(2)).to(self.device)

            y_tm1 = torch.tensor([self.tgt_vocab[hyp[-1]] for hyp in hypotheses], dtype=torch.long, device=self.device)
            y_t_embed = self.target_embedding(y_tm1)

            x = torch.cat([y_t_embed, att_tm1], dim=-1)

            h_t, att_t = self.decoder.step(x, h_tm1,
                                exp_src_encodings, exp_src_encodings_att_linear)

            h_t, c_t = h_t

            # log probabilities over target words
            log_p_t = F.log_softmax(self.decoder.target_vocab_projection(att_t), dim=-1)

            live_hyp_num = beam_size - len(completed_hypotheses)
            contiuating_hyp_scores = (hyp_scores.unsqueeze(1).expand_as(log_p_t) + log_p_t).view(-1)
            top_cand_hyp_scores, top_cand_hyp_pos = torch.topk(contiuating_hyp_scores, k=live_hyp_num)

            prev_hyp_ids = torch.div(top_cand_hyp_pos, len(self.tgt_vocab), rounding_mode='floor')
            hyp_word_ids = top_cand_hyp_pos % len(self.tgt_vocab)

            new_hypotheses = []
            live_hyp_ids = []
            new_hyp_scores = []

            for prev_hyp_id, hyp_word_id, cand_new_hyp_score in zip(prev_hyp_ids, hyp_word_ids, top_cand_hyp_scores):
                prev_hyp_id = prev_hyp_id.item()
                hyp_word_id = hyp_word_id.item()
                cand_new_hyp_score = cand_new_hyp_score.item()

                hyp_word = self.tgt_vocab.id2word[hyp_word_id]
                new_hyp_sent = hypotheses[prev_hyp_id] + [hyp_word]
                if hyp_word == '</s>':
                    completed_hypotheses.append(Hypothesis(value=new_hyp_sent[1:-1],
                                                           score=cand_new_hyp_score))
                else:
                    new_hypotheses.append(new_hyp_sent)
                    live_hyp_ids.append(prev_hyp_id)
                    new_hyp_scores.append(cand_new_hyp_score)

            if len(completed_hypotheses) == beam_size:
                break

            live_hyp_ids = torch.tensor(live_hyp_ids, dtype=torch.long, device=self.device)
            h_tm1 = h_t[live_hyp_ids], c_t[live_hyp_ids]
            att_tm1 = att_t[live_hyp_ids]

            hypotheses = new_hypotheses
            hyp_scores = torch.tensor(new_hyp_scores, dtype=torch.float, device=self.device)

        if len(completed_hypotheses) == 0:
            completed_hypotheses.append(Hypothesis(value=hypotheses[0][1:],
                                                   score=hyp_scores[0].item()))

        completed_hypotheses.sort(key=lambda hyp: hyp.score, reverse=True)

        return completed_hypotheses

    @staticmethod
    def load(model_path: str):
        """ Load the model from a file.
        @param model_path (str): path to model
        """
        params = torch.load(model_path, map_location=lambda storage, loc: storage)
        args = params['args']
        model = SRL(
            src_vocab = params['vocab']['source'],
            tgt_vocab = params['vocab']['target'],
            device = params['device'],
            **args
        )
        model.load_state_dict(params['state_dict'])
        return model

    def save(self, path: str):
        """ Save the odel to a file.
        @param path (str): path to the model
        """
        print('save model parameters to [%s]' % path, file=sys.stderr)

        params = {
            'args': dict(embed_size=self.embed_size, hidden_size=self.hidden_size),
            'vocab': dict(source=self.src_vocab, target=self.tgt_vocab),
            'state_dict': self.state_dict(),
            'device': self.device
        }
        torch.save(params, path)

In [43]:
def batch_iter(data, batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    :param data: list of tuples containing source and target sentence. ie.
        (list of (src_sent, tgt_sent))
    :type data: List[Tuple[List[str], List[str]]]
    :param batch_size: batch size
    :type batch_size: int
    :param shuffle: whether to randomly shuffle the dataset
    :type shuffle: boolean
    """
    batch_num = math.ceil(len(data) / batch_size)
    index_array = list(range(len(data)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size]
        examples = [data[idx] for idx in indices]

        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
        src_sents = [e[0] for e in examples]
        tgt_sents = [e[1] for e in examples]

        print(f'batch size is {batch_size}')
        print(len(src_sents))

        yield src_sents, tgt_sents

In [44]:
def evaluate_ppl(model, val_data, batch_size=32):
    """ Evaluate perplexity on dev sentences
    :param model: SRL Model
    :type model: SRL
    :param dev_data: list of tuples containing source and target sentence.
        i.e. (list of (src_sent, tgt_sent))
    :param val_data: List[Tuple[List[str], List[str]]]
    :param batch_size: size of batches to extract
    :type batch_size: int
    :returns ppl: perplixty on val sentences
    """
    was_training = model.training
    model.eval()

    cum_loss = 0.
    cum_tgt_words = 0.

    # no_grad() signals backend to throw away all gradients
    with torch.no_grad():
        for src_sents, tgt_sents in batch_iter(val_data, batch_size):
            loss = -model(src_sents, tgt_sents).sum()

            cum_loss += loss.item()
            tgt_word_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
            cum_tgt_words += tgt_word_num_to_predict

        ppl = np.exp(cum_loss / cum_tgt_words)

    if was_training:
        model.train()

    return ppl

In [45]:
def train_and_evaluate(model, train_data, val_data, optimizer, train_batch_size=32, clip_grad=2, log_every = 100,
                       valid_niter = 500, model_save_path="srl.ckpt", num_epoch=6):
    num_trail = 0
    cum_examples = report_examples = epoch = valid_num = 0
    hist_valid_scores = []
    train_iter = patience = cum_loss = report_loss = cum_tgt_words = report_tgt_words = 0

    print('begin Maximum Likelihood training')
    train_time = begin_time = time.time()

    val_data_tgt = [tgt for _, tgt in val_data]

    for epoch in tqdm(range(num_epoch)):
        for src_sents, tgt_sents in batch_iter(train_data, batch_size=train_batch_size, shuffle=True):
            train_iter += 1

            optimizer.zero_grad()

            batch_size = len(src_sents)

            example_losses = -model(src_sents, tgt_sents)
            batch_loss = example_losses.sum()
            loss = batch_loss / batch_size
            loss.backward()

            # clip gradient
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

            optimizer.step()

            batch_losses_val = batch_loss.item()
            report_loss += batch_losses_val
            cum_loss += batch_losses_val

            tgt_words_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
            report_tgt_words += tgt_words_num_to_predict
            cum_tgt_words += tgt_words_num_to_predict
            report_examples += batch_size
            cum_examples += batch_size

            if train_iter % log_every == 0:
                print('epoch %d, iter %d, avg. loss %.2f, avg. ppl %.2f ' \
                        'cum. examples %d, speed %.2f words/sec, time elapsed %.2f sec' % (epoch, train_iter,
                                                                                            report_loss / report_examples,
                                                                                            math.exp(report_loss / report_tgt_words),
                                                                                            cum_examples,
                                                                                            report_tgt_words / (time.time() - train_time),
                                                                                            time.time() - begin_time))
                train_time = time.time()
                report_loss = report_tgt_words = report_examples = 0.



            # perform validation
            if train_iter % valid_niter == 0:
                print('epoch %d, iter %d, cum. loss %.2f, cum. ppl %.2f cum. examples %d' % (epoch, train_iter,
                                                                                            cum_loss / cum_examples,
                                                                                            np.exp(cum_loss / cum_tgt_words),
                                                                                            cum_examples))

                cum_loss = cum_examples = cum_tgt_words = 0.
                valid_num += 1

                print('begin validation ...', file=sys.stderr)

                # compute dev. ppl
                dev_ppl = evaluate_ppl(model, val_data, batch_size=128)   # dev batch size can be a bit larger
                valid_metric = -dev_ppl


                print('validation: iter %d, dev. ppl %f' % (train_iter, dev_ppl), file=sys.stderr)

                is_better = len(hist_valid_scores) == 0 or valid_metric > max(hist_valid_scores)
                hist_valid_scores.append(valid_metric)

                if is_better:
                    print('save currently the best model to [%s]' % model_save_path, file=sys.stderr)
                    model.save(model_save_path)

                    # also save the optimizers' state
                    torch.save(optimizer.state_dict(), model_save_path + '.optim')


In [50]:
def convert_output(biolst,input, predict_output,tag):
  word_to_idx = {word:idx for idx, word  in enumerate(input)}
  all_idx = []
  for w in predict_output:
    if w in word_to_idx:
      all_idx.append(word_to_idx[w])
  all_idx.sort()
  prev = None
  for idx in all_idx:
    if prev and prev == idx-1:
      biolst[idx] = "I-"+tag
    else:
      biolst[idx] = "B-"+tag
    prev = idx
  return biolst

In [51]:
#use beam search to generate output
val_data_src = [src for src, _ in val_data]
val_pred = []

t = len(val_data_src)//5
for i in range(t):
  input = val_data_src[i*5][2:-1]
  BIOlst_output = ['O'] * len(input)
  for j in range(5):
    pos = i*5+j
    s = val_data_src[pos]
    tag = s[-1]
    result = model.beam_search(
                  s,
                  beam_size=16,
                  max_decoding_time_step=len(s)
              )
    pred = result[0].value
    input = s[2:-1]
    BIOlst_output = convert_output(BIOlst_output, input, pred,tag)
  val_pred.append(BIOlst_output)


In [52]:
val_pred_2 = []
for i in val_pred:
  val_pred_2.extend(i)
y_pred_model2_dict = format_output_labels(val_pred_2, val_idx)
y_true_model2_dict = format_output_labels(val_true, val_idx)
print(str(mean_f1(y_pred_model2_dict, y_true_model2_dict)))

0.22302378310024523
